In [1]:
import requests
from bs4 import BeautifulSoup
import os

<h3>Create Dataset from Rap Genius Data</h3>

In [4]:
with open("../api-key.txt", "r") as file:
    access_token = file.read()
base_url = "http://api.genius.com"
headers = {'Authorization': f'Bearer {access_token}'}
search_url = base_url + "/search"
song_title = "Lithium"
params = {'q': song_title}
response = requests.get(search_url, params=params, headers=headers)
response.status_code

200

In [6]:
data = response.json()['response']['hits']
for idx, i in enumerate(data):
    print(data[idx]['result']['title'], "----", data[idx]['result']['primary_artist']['name'])
    print(data[idx]["result"]["api_path"])
    print()

Lithium ---- Nirvana
/songs/56433

Lithium ---- Evanescence
/songs/183356

Lithium ---- Trippie Redd
/songs/3454487

Lithium ---- BONES
/songs/4524735

Black Lithium ---- Canibus
/songs/3795666

Lithium (Synthesis) ---- Evanescence
/songs/3235559

Lithium ---- Method Man
/songs/3945651

Lithium Sunset ---- Sting
/songs/389349

Lithium Lips ---- Mac Lethal
/songs/151212

Lithium ---- Muse
/songs/766569



In [11]:
headers = {'Authorization': f'Bearer {access_token}'}
lyric_query = base_url + "/referents"

params = {'song_id': "56433"}
response = requests.get(lyric_query, params=params, headers=headers)
response.json()['response']['referents'][0]['fragment']

'I’m so lonely, that’s okay, I shaved my head'

In [12]:
for referent in response.json()['response']['referents']:
    print(referent["id"])
    print(referent['fragment'])
    print(referent['annotations'][0]['body']['dom']['children'][0]['children'])
    print()

17701902
I’m so lonely, that’s okay, I shaved my head
['As far as anyone knows, Kurt Cobain has never actually shaved his head and made this line up for the purpose of the song.']

11055484
And I’m not sad
['He uses a negative phrase like “I’m not sad” rather than “I am happy” or something positive. This puts the word “sad” in the listener’s head, and conveys a more depressed meaning than an uplifting one.']

7619144
Yeah, yeah
 Yeah, yeah
 Yeah, yeah
 Yeah, yeah
 Yeah, yeah
 Yeah, yeah
 Yeah
['The chorus also contrasts the verses as to illustrate the contrast between mania and depression in the Bipolar mind.']

1970348
I’m so happy ’cause today I found my friends, they’re in my head
['This song is about a man becoming religious to get over the death of his spouse, so I would suggest that these lines set the tone of that story. He’s happy because he has finally found friends in his head and this makes him happy because it is allowing him to finally get over his depression of his wife d